In [ ]:
import random
import cv2
import numpy as np
from PIL import Image
import io
import base64
import requests
import ipywidgets as widgets
from IPython.display import display as ipy_display
from IPython.display import HTML
from google.colab import output
from google.colab.patches import cv2_imshow

class Memory_Game():
    def __init__(self):
        self.memory_grid_space = [(i,j) for i in range(4) for j in range(4)]
        self.memory_matrix = [[0 for i in range(4)] for j in range(4)]
        self.click_count = 0
        self.message = 'Please select any grid'
        self.prev_grid_img = 0
        self.prev_selected_grid_no = 0
        self.solved_grid = 0
        # self.test_img = cv2.imread('test9.png')
        self.test_img = self.get_image()
        self.test_img = cv2.resize(self.test_img, (400,400))
        self.videoWriter = self.create_videoWriter('output_video.mp4')

    def start_game(self):
        self.generate_memory_grid()
        self.grid_img = self.get_empty_grid_img()
        self.get_click_coordinates()
        # self.display_clickable_image()

    def generate_memory_grid(self):
        random.shuffle(self.memory_grid_space)
        for num in range(1,9):
            random_grid_spaces = random.sample(self.memory_grid_space, 2)
            for grid_space in random_grid_spaces:
                self.memory_matrix[grid_space[0]][grid_space[1]] = num
                self.memory_grid_space.remove(grid_space)
        return self.memory_matrix

    def get_image(self):
      # File ID from the Google Drive link
      url = f'https://drive.google.com/uc?export=download&id=1D91UYKoobRN9Hh8Jqp_nz3i0vlyq3ehj'
      response = requests.get(url)

      if response.status_code == 200:
          img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
          img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
          return img
      else:
          return cv2.imread('test9.png')

    def create_videoWriter(self,output_video_path):
      fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' or 'H264' for MP4 format
      fps = 3  # Frames per second, adjust as needed
      video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (400,400))
      return video_writer

    def onclick_change_grid(self, coord):
        row = coord[0]//100
        col = coord[1]//100

        if (row,col) not in self.memory_grid_space:
            # Calculate the center of each grid cell
            x = col * 100 + 25
            y = row * 100 + 70
            text = f"{self.memory_matrix[row][col]}"
            self.memory_grid_space.append((row,col))
            self.grid_img = cv2.putText(self.grid_img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)

        elif(self.prev_selected_grid_no>0):
            x1,x2,y1,y2 = self.get_coordinates_of_grid(row,col)
            px1,px2,py1,py2 = self.prev_grid_coord
            if(self.prev_selected_grid_no == self.memory_matrix[row][col]):
                if(x1 != px1 or x2!=px2 or y1!=py1 or y2!=py2):
                    self.grid_img[y1:y2,x1:x2,:] = self.test_img[y1:y2,x1:x2,:]
                    self.grid_img[py1:py2,px1:px2,:] = self.test_img[py1:py2,px1:px2,:]
                    self.message = 'Please select any grid'
                    self.prev_selected_grid_no = 0
                    self.solved_grid+=1
                    if(len(self.memory_grid_space)==16 and self.solved_grid==8):
                        self.videoWriter.write(self.grid_img)
                        print('Congrates You Own the Game')
                        print(f"Your game succesfully saved to 'output_video.mp4'")
                        self.videoWriter.release()
                        cv2.destroyAllWindows()
                        return
                else:
                    self.message = 'Hey!! U have selected same grid. Please select another grid of ' + str(self.prev_selected_grid_no)
            else:
                self.grid_img[py1:py2,px1:px2,:] = self.prev_grid_img.copy()
                self.present_seleted_grid(row,col)
                self.message = 'Please select another grid of ' + str(self.prev_selected_grid_no)

        else:
            self.present_seleted_grid(row,col)
            self.message = 'Please select another grid of ' + str(self.prev_selected_grid_no)
        self.get_click_coordinates()

    def present_seleted_grid(self,row,col):
        x1,x2,y1,y2 = self.get_coordinates_of_grid(row,col)
        self.prev_grid_img = self.grid_img[y1:y2,x1:x2,:].copy()
        self.grid_img[y1:y2,x1:x2,1] = 255
        self.prev_selected_grid_no = self.memory_matrix[row][col]
        self.prev_grid_coord = (x1,x2,y1,y2)
        self.grid_solved = False

    def get_coordinates_of_grid(self,row,col):
        x1 = col * 100
        y1 = row * 100
        x2 = (col+1) * 100
        y2 = (row+1) * 100
        return x1,x2,y1,y2

    def get_empty_grid_img(self):
        test_img = np.ones((400,400, 3), dtype=np.uint8) * 255
        # draw horizontal lines
        for i in range(11):
            test_img = cv2.line(test_img, (0, i * 100), (400, i * 100), (255, 0, 0), 2)
        # draw vertical lines
        for i in range(11):
            test_img = cv2.line(test_img, (i * 100, 0), (i * 100, 400), (255, 0, 0), 2)
        return test_img

    def convert_image_to_base64(self,image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
        buffer = io.BytesIO()
        image_pil.save(buffer, format="PNG")
        img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
        return img_base64

    # Step 3: Add JavaScript code to capture mouse clicks and send coordinates
    def get_click_coordinates(self):
        self.videoWriter.write(self.grid_img)
        img_base64 = self.convert_image_to_base64(self.grid_img)
        self.click_count+=1

        html_content = f'''
            <div id="clickable_container_{self.click_count}">
                <p id="showMessage_{self.click_count}">{self.message}</p>
                <img src="data:image/png;base64,{img_base64}" id="clickable_image_{self.click_count}" style="max-width:400px; max-height: 400px;">
            </div>
            <script>
                const image_{self.click_count} = document.getElementById("clickable_image_{self.click_count}");

                image_{self.click_count}.onclick = function(event) {{
                    const rect = image_{self.click_count}.getBoundingClientRect();
                    const x = event.clientX - rect.left;
                    const y = event.clientY - rect.top;

                    // Clear the previous HTML content
                    const previousContainer = document.getElementById("clickable_container_{self.click_count}");
                    if (previousContainer) {{
                        previousContainer.remove();
                    }}

                    // Send the grid coordinates back to Python
                    google.colab.kernel.invokeFunction('notebook.game_on_click', [y,x], {{}});

                }};
            </script>
        '''

        # Display the image with embedded JavaScript for interactivity
        ipy_display(HTML(html_content))


memory_game = Memory_Game()
memory_game.start_game()

# Define the function to handle the clicked coordinates from JS
def game_on_click(x, y):
    memory_game.onclick_change_grid([x, y])

# Register the function so it can be called from JavaScript
output.register_callback('notebook.game_on_click', game_on_click)

In [ ]:
# Display video in google colab
import IPython.display as ipd
ipd.Video('output_video.mp4', width=400,height=400,embed=True)